# Import all the libraries

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
import gc
import os
import tensorflow as tf
from openslide import open_slide
import keras
from openslide.deepzoom import DeepZoomGenerator
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import applications


In [2]:
inp_size=512
eff_res_stacked = '/kaggle/input/all-models-final/effb0_res151_stacked.h5'
res='/kaggle/input/all-models-final/Resnet152.h5'
eff='/kaggle/input/all-models-final/effB0.h5'

simple_cnn = keras.Sequential([
    layers.Conv2D(filters = 8, kernel_size = (3,3),input_shape = (inp_size,inp_size,3), padding = 'same', activation = 'relu'),
    layers.MaxPool2D(pool_size=2,strides=2),
    layers.Conv2D(filters = 16, kernel_size = (3,3), padding = 'same', activation = 'relu'),
    layers.MaxPool2D(pool_size=2,strides=2),
    layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(2,activation='softmax'),
])

# input_layer = layers.Input(shape = (inp_size, inp_size, 3))
# base_1 = keras.applications.EfficientNetB0(weights = 'imagenet', include_top = False, input_shape = (inp_size, inp_size, 3))
# for layer in base_1.layers:
#     layer.trainable =  False
# model_1 = base_1(input_layer)
# model_1 = layers.GlobalAveragePooling2D()(model_1)
# output_1 = layers.Flatten()(model_1)
# x = layers.BatchNormalization()(output_1)
# x = layers.Dense(512,activation = 'relu')(x)
# x = layers.Dropout(0.25)(x)
# x = layers.Dense(128,activation = 'relu')(x)
# x = layers.Dense(2,activation='softmax')(x)
# eff = tf.keras.models.Model(inputs = input_layer, outputs = x)


# #---------------
# input_layer = layers.Input(shape = (inp_size, inp_size, 3))
# base_1 = keras.applications.EfficientNetB0(weights = 'imagenet', include_top = False, input_shape = (inp_size, inp_size, 3))
# for layer in base_1.layers:
#     layer.trainable =  False
# model_1 = base_1(input_layer)
# model_1 = layers.GlobalAveragePooling2D()(model_1)
# output_1 = layers.Flatten()(model_1)
# x = layers.BatchNormalization()(output_1)
# x = layers.Dense(128,activation = 'relu')(x)
# x = layers.Dense(2,activation='softmax')(x)
# eff_single_fc = tf.keras.models.Model(inputs = input_layer, outputs = x)

2023-02-01 18:33:05.959879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 18:33:06.084071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 18:33:06.084868: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 18:33:06.086638: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

# Hyper Parameters to be Tuned

In [3]:
models=[]
model_list = []
models.append(simple_cnn)
epochs=30
batch_size=32
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  #adam or sgd

color_normalized = 'no'

# Fixed Parameters and paths

In [4]:
if color_normalized == 'yes':
    dataset_directory='/kaggle/input/mayo-512720-dataset/DATASET/norm'
else:
    dataset_directory='/kaggle/input/mayo-512720-dataset/DATASET/original'

#----------------------
classMode = 'categorical'
loss = 'categorical_crossentropy'
val_split = 0
Num_tiles_to_consider_for_prediction = 3
validation_steps= 10
steps_per_epoch = 10
val_batch_size=16

train_csv='/kaggle/input/mayo-clinic-strip-ai/train.csv'
test_csv = '/kaggle/input/mayo-clinic-strip-ai/test.csv'
sample_sub_csv='/kaggle/input/mayo-clinic-strip-ai/sample_submission.csv'

train_imgs_dir = '/kaggle/input/mayo-clinic-strip-ai/train'
test_imgs_dir='/kaggle/input/mayo-clinic-strip-ai/test'

# Load and compile the models

In [5]:
for i in model_list:
    models.append(keras.models.load_model(i))

In [6]:
for i in models:
    i.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy'])

In [7]:
models

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45, 
    validation_split=val_split,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.5,2.0],
    zoom_range = [0.5,2.0],
    horizontal_flip=True,
    vertical_flip=True
)

print(dataset_directory)
train_generator = datagen.flow_from_directory(
    dataset_directory,
    target_size=(inp_size, inp_size),
    batch_size=batch_size,
    shuffle=True,
    class_mode=classMode,
    subset='training',
)

validation_generator = datagen.flow_from_directory(
    dataset_directory,
    target_size=(inp_size, inp_size),
    batch_size=val_batch_size,
    shuffle=True,
    class_mode=classMode,
    subset='validation',
)

class_indices = train_generator.class_indices
print(class_indices)

/kaggle/input/mayo-512720-dataset/DATASET/original
Found 4871 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
{'ce': 0, 'laa': 1}


In [9]:
for i in models:
    i.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        epochs=epochs,
#         validation_data=validation_generator,
    )    

2023-02-01 18:33:16.490973: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2023-02-01 18:33:21.312787: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


10/10 [==============================] - 36s 3s/step - loss: 11.1823 - accuracy: 0.4625
Epoch 2/30
10/10 [==============================] - 31s 3s/step - loss: 1.3226 - accuracy: 0.5562
Epoch 3/30
10/10 [==============================] - 29s 3s/step - loss: 0.7235 - accuracy: 0.5492
Epoch 4/30
10/10 [==============================] - 30s 3s/step - loss: 0.7000 - accuracy: 0.4875
Epoch 5/30
10/10 [==============================] - 30s 3s/step - loss: 0.6877 - accuracy: 0.5750
Epoch 6/30
10/10 [==============================] - 30s 3s/step - loss: 0.6940 - accuracy: 0.5031
Epoch 7/30
10/10 [==============================] - 30s 3s/step - loss: 0.6906 - accuracy: 0.5219
Epoch 8/30
10/10 [==============================] - 29s 3s/step - loss: 0.6885 - accuracy: 0.5469
Epoch 9/30
10/10 [==============================] - 29s 3s/step - loss: 0.6872 - accuracy: 0.5437
Epoch 10/30
10/10 [==============================] - 29s 3s/step - loss: 0.6812 - accuracy: 0.5719
Epoch 11/30
10/10 [==========

# model testing

In [10]:
def make_test_file(x):
    return os.path.join(test_imgs_dir,x+'.tif')
test = pd.read_csv(test_csv)
test_data = pd.DataFrame({'image_id': test.image_id.apply(make_test_file)})
test_data.head()

,image_id
0,/kaggle/input/mayo-clinic-strip-ai/test/006388...
1,/kaggle/input/mayo-clinic-strip-ai/test/008e5c...
2,/kaggle/input/mayo-clinic-strip-ai/test/00c058...
3,/kaggle/input/mayo-clinic-strip-ai/test/01adc5...


In [11]:
preds=[]
for x in range(int(test_data.size)):
    img_path = test_data.image_id[x]
    slide = open_slide(img_path)
    tiles=DeepZoomGenerator(slide,tile_size=inp_size,overlap=0,limit_bounds=False)
    cols,rows = tiles.level_tiles[tiles.level_count-1]
    print(x)
    temp_preds=[]
    count=0
    
    for row in range(0,rows,5):
        for col in range(0,cols,5):
            tile=tiles.get_tile(tiles.level_count-1,(col,row))
            tile=tile.convert("RGB")
            tile=np.array(tile)
            try:
                if tile.mean()<180 and tile.std()>50:                    
                    tile = np.reshape(tile, [1,inp_size, inp_size, 3])
                    p=[i.predict(tile/255) for i in models]
                    t_p = sum(p)/len(p)                    
                    temp_preds.append(t_p)
                    count+=1
                    if count>Num_tiles_to_consider_for_prediction:break
            except :
                pass        
            if count>Num_tiles_to_consider_for_prediction:break
    if len(temp_preds) > 0:
        preds.append(sum(temp_preds)/len(temp_preds))
    else:
        preds.append([[0.5,0.5]])
    del slide
    del tiles
    gc.collect()

0
1
2
3


In [12]:
preds

[array([[0.5499059 , 0.45009407]], dtype=float32),
 array([[0.52042913, 0.4795708 ]], dtype=float32),
 array([[0.6839167 , 0.31608328]], dtype=float32),
 array([[0.5301441 , 0.46985596]], dtype=float32)]

In [13]:
preds = pd.DataFrame(np.concatenate(preds))
submission = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/sample_submission.csv')
submission.CE = preds.iloc[ : , : 1]
submission.LAA = preds.iloc[ : , 1: 2]
submission = submission.groupby("patient_id").mean()
submission = submission[["CE", "LAA"]].round(6).reset_index()
submission.fillna(0.5)
submission

,patient_id,CE,LAA
0,006388,0.549906,0.450094
1,008e5c,0.520429,0.479571
2,00c058,0.683917,0.316083
3,01adc5,0.530144,0.469856


In [14]:
submission[["patient_id", "CE", "LAA"]].to_csv("submission.csv", index=False)
!head submission.csv

patient_id,CE,LAA
006388,0.549906,0.450094
008e5c,0.520429,0.479571
00c058,0.683917,0.316083
01adc5,0.530144,0.469856
